In [20]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [95]:
# get_matches

import re

def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    Winner = []
    Score = []
    Loser = []
    Goals = []

    for match in matches:
        Winner.append(match.find('th', class_='fhome').get_text())
        Score.append(match.find('th', class_='fscore').get_text())
        Loser.append(match.find('th', class_='faway').get_text())
        
        #delete '\n', Report and Reference
        str1= match.find('tr', class_='fgoals').get_text().replace('\n', ' ').replace('Report', ' ')
        str1= re.sub('\[.*?\]','',str1)
        Goals.append(str1)

    dict_football = {'Winner': Winner, 'Score': Score, 'Loser': Loser, 'Goals':Goals}
    df_football = pd.DataFrame(dict_football)
    
    df_football.insert(loc=0, column='Year', value=year) 
   
    return df_football

In [96]:
# data to scraper -> World_cups 1930-2022
years = range(1930, 2023, 4)

# results: historical data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)

In [97]:
df_fifa

,Year,Winner,Score,Loser,Goals
0,1930,France,4–1,Mexico,"L. Laurent 19'Langiller 40'Maschinot 43', ..."
1,1930,Argentina,1–0,France,Monti 81'
2,1930,Chile,3–0,Mexico,"Vidal 3', 65'M. Rosas 52' (o.g.)"
3,1930,Chile,1–0,France,Subiabre 67'
4,1930,Argentina,6–3,Mexico,"Stábile 8', 17', 80'Zumelzú 12', 55'Varallo..."
...,...,...,...,...,...
660,2022,England,1–2,France,Kane 54' (pen.) Tchouaméni 17' Giroud ...
661,2022,Argentina,Match 61,Croatia,
662,2022,France,Match 62,Morocco,
663,2022,Losers Match 61,Match 63,Losers Match 62,


In [100]:
df_fifa.to_csv("fifa_worldcup_data_1930-2022.csv", encoding='utf-8-sig', index=False)